<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/Peoplewalking_Range_Doppler_and_Spectrogram_andWavelet_BiLSTM_twoclass_mixup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #Burada dataset 1 range doppler , spectrogram ve wavelet bi-lstm encoder ile train edildi, augmentation yok

# 70 epoch koşuldu
#test_accuracy_per_run[0.8154150247573853, 0.8332015752792359, 0.7802371621131897]
# f1_score_per_run[0.7872003610424663, 0.814550594333203, 0.8011259938318762]
#Mean test accuracy is 0.810, mean test f1 score is 0.801, max test accuracy is 0.833, max test f1 score is 0.815, min test accuracy is 0.780, min test f1 score is 0.787, std of test accuracy is 0.022, std of test f1 score is 0.011
#Time elapsed through all process: 376.589, sec

In [1]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

# Load Range-Doppler fast data
range_doppler_fast = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast.mat')
range_doppler_fast = range_doppler_fast['range_doppler_fast']
range_doppler_fast = np.transpose(range_doppler_fast, (2, 0, 1)) # label x width x height
range_doppler_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast_label.mat')
range_doppler_fast_label = range_doppler_fast_label['range_doppler_fast_label']

# Load Range-Doppler slow data

range_doppler_slow = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow.mat')
range_doppler_slow = range_doppler_slow['range_doppler_slow']
range_doppler_slow = np.transpose(range_doppler_slow, (2, 0, 1))# label x width x height
range_doppler_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow_label.mat')
range_doppler_slow_label = range_doppler_slow_label['range_doppler_slow_label']

# Load Spectrogram fast data
spectrogram_fast_resized = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_resized.mat')
spectrogram_fast_resized = spectrogram_fast_resized['spectrogram_fast_resized']
spectrogram_fast_resized = np.transpose(spectrogram_fast_resized, (2, 0, 1)) # label x width x height
spectrogram_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_label.mat')
spectrogram_fast_label = spectrogram_fast_label['spectrogram_fast_label']


# Load Spectrogram slow data
spectrogram_slow_resized= scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_resized.mat')
spectrogram_slow_resized = spectrogram_slow_resized['spectrogram_slow_resized']
spectrogram_slow_resized = np.transpose(spectrogram_slow_resized, (2, 0, 1)) # label x width x height
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']


# Load Wavelet fast data
wavelet_fast = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/wavelet_fast.mat')
wavelet_fast = wavelet_fast['wavelet_fast']
wavelet_fast= np.transpose(wavelet_fast, (2, 0, 1)) # label x width x height
wavelet_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/wavelet_fast_label.mat')
wavelet_fast_label = wavelet_fast_label['wavelet_fast_label']

# Load Wavelet slow data
wavelet_slow = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/wavelet_slow.mat')
wavelet_slow = wavelet_slow['wavelet_slow']
wavelet_slow = np.transpose(wavelet_slow, (2, 0, 1)) # label x width x height
wavelet_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/wavelet_slow_label.mat')
wavelet_slow_label = wavelet_slow_label['wavelet_slow_label']



In [4]:
# check if there is a NaN and check limit values

import numpy as np

X_train=range_doppler_fast
y_train=range_doppler_fast_label
print("Input NaNs:", np.isnan(X_train).any())
print("Label NaNs:", np.isnan(y_train).any())

print("Input max/min:", np.max(X_train), np.min(X_train))
print("Label max/min:", np.max(y_train), np.min(y_train))


X_train=range_doppler_slow
y_train=range_doppler_slow_label
print("Input NaNs:", np.isnan(X_train).any())
print("Label NaNs:", np.isnan(y_train).any())

print("Input max/min:", np.max(X_train), np.min(X_train))
print("Label max/min:", np.max(y_train), np.min(y_train))


X_train=spectrogram_fast_resized
y_train=spectrogram_fast_label
print("Input NaNs:", np.isnan(X_train).any())
print("Label NaNs:", np.isnan(y_train).any())

print("Input max/min:", np.max(X_train), np.min(X_train))
print("Label max/min:", np.max(y_train), np.min(y_train))

X_train=spectrogram_slow_resized,
y_train=spectrogram_slow_label
print("Input NaNs:", np.isnan(X_train).any())
print("Label NaNs:", np.isnan(y_train).any())

print("Input max/min:", np.max(X_train), np.min(X_train))
print("Label max/min:", np.max(y_train), np.min(y_train))



X_train=wavelet_fast
y_train=wavelet_fast_label
print("Input NaNs:", np.isnan(X_train).any())
print("Label NaNs:", np.isnan(y_train).any())

print("Input max/min:", np.max(X_train), np.min(X_train))
print("Label max/min:", np.max(y_train), np.min(y_train))

X_train=wavelet_slow,
y_train=wavelet_slow_label
print("Input NaNs:", np.isnan(X_train).any())
print("Label NaNs:", np.isnan(y_train).any())

print("Input max/min:", np.max(X_train), np.min(X_train))
print("Label max/min:", np.max(y_train), np.min(y_train))






Input NaNs: False
Label NaNs: False
Input max/min: 162.23370093799332 90.08805240827945
Label max/min: 1 1
Input NaNs: False
Label NaNs: False
Input max/min: 161.23241044097242 90.42134811782459
Label max/min: 0 0
Input NaNs: False
Label NaNs: False
Input max/min: 0.9490114091247386 0.32231198587013316
Label max/min: 1 1
Input NaNs: False
Label NaNs: False
Input max/min: 0.9516653596667601 0.30782453795818054
Label max/min: 0 0
Input NaNs: False
Label NaNs: False
Input max/min: 1314.9241134362467 -2.65045638865838
Label max/min: 1 1
Input NaNs: False
Label NaNs: False
Input max/min: 1601.1927695176032 -2.9783127546767716
Label max/min: 0 0


In [5]:


# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_fast,range_doppler_slow),axis=0) # fast =fall, slow=walking
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis]
range_doppler_concat_label = np.zeros((range_doppler_concat.shape[0],1))
range_doppler_concat_label[:range_doppler_fast.shape[0],:] = 1
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]


# Concat spectrogram data
spectrogram_concat = np.concatenate((spectrogram_fast_resized,spectrogram_slow_resized),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]
spectrogram_concat_label = np.zeros((spectrogram_concat.shape[0],1))
spectrogram_concat_label[:spectrogram_fast_resized.shape[0],:] = 1
# Shuffle concat spectrogram
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

# Concat wavelet data
wavelet_concat = np.concatenate((wavelet_fast,wavelet_slow),axis=0)
wavelet_concat = wavelet_concat[:,:,:,np.newaxis]
wavelet_concat_label = np.zeros((wavelet_concat.shape[0],1))
wavelet_concat_label[:wavelet_fast.shape[0],:] = 1

# Shuffle concat wavelet
wavelet_concat_shuffle = wavelet_concat[shuffle_indx,:,:,:]
wavelet_concat_label_shuffle = wavelet_concat_label[shuffle_indx,:]

In [6]:
def mixup_augmentation(images,range_doppler_training_data,wavelet_training_data, labels, repeat_of_mixup, alpha=0.2):
    batch_size = images.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],\
                                            range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))

    concat_images_wavelet = np.zeros((batch_size*(repeat_of_mixup+1),wavelet_training_data.shape[1],\
                                            wavelet_training_data.shape[2],wavelet_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))




    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_1 = images[shuffle_indx_1,:,:,:]
      range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      wavelet_training_data_shuffled_1 = wavelet_training_data[shuffle_indx_1,:,:,:]


      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_2 = images[shuffle_indx_2,:,:,:]
      range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      wavelet_training_data_shuffled_2 = wavelet_training_data[shuffle_indx_2,:,:,:]

      labels_shuffled_2 = labels[shuffle_indx_2,:]

      # Sample lambda and reshape it to do the mixup
     #  gaussian_mean = 0.2
      # gaussian_std = 0.02
      # ll = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
      alpha=0.2
      ll=np.random.beta(alpha,alpha, (batch_size,1,1,1))

      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))

      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup = images_shuffled_1 * x_l + images_shuffled_2 * (1 - x_l)
      images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
      images_mixup_wavelet = wavelet_training_data_shuffled_1 * x_l + wavelet_training_data_shuffled_2 * (1 - x_l)

      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)
      concat_images[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_images_wavelet[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_wavelet

      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images[repeat_of_mixup*batch_size:,:,:,:] = images
    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
    concat_images_wavelet[repeat_of_mixup*batch_size:,:,:,:] = wavelet_training_data

    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images,concat_images_range_doppler, concat_images_wavelet,concat_label)






def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,wavelet_concat_shuffle_train,label_shuffle_train):

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx

  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  wavelet_validation_data = wavelet_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]

  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  wavelet_training_data = wavelet_concat_shuffle_train[randomlist_for_train_indx,:,:,:]


  #get labels
  validation_labels = label_shuffle_train[randomlist_for_validation_indx,:]
  training_labels = label_shuffle_train[randomlist_for_train_indx,:]


  # get CLASS1 and CLASS2 indexes of training dat

  class1_indexes = np.where(training_labels == 0)[0]
  class2_indexes = np.delete(range(0, training_labels.shape[0]), class1_indexes)


  class1_spectrograms_train = spectrogram_training_data[class1_indexes,:,:,:]
  class2_spectrograms_train = spectrogram_training_data[class2_indexes,:,:,:]


  class1_rangedoppler_train = range_doppler_training_data[class1_indexes,:,:,:]
  class2_rangedoppler_train = range_doppler_training_data[class2_indexes,:,:,:]

  class1_wavelet_train = wavelet_training_data[class1_indexes,:,:,:]
  class2_wavelet_train = wavelet_training_data[class2_indexes,:,:,:]

  size_of_samples_class1= class1_spectrograms_train.shape[0]
  size_of_samples_class2 = class2_spectrograms_train.shape[0]


  class2_label = np.ones((size_of_samples_class2,1))
  class1_label = np.zeros((size_of_samples_class1,1))


  # unify data
  spectrogram_training_data = np.concatenate((class2_spectrograms_train,class1_spectrograms_train),axis=0)
  range_doppler_training_data = np.concatenate((class2_rangedoppler_train,class1_rangedoppler_train),axis=0)
  wavelet_training_data = np.concatenate((class2_wavelet_train,class1_wavelet_train),axis=0)


  training_labels = np.concatenate((class2_label,class1_label),axis=0)


  (spectrogram_augmented_image,range_doppler_augmented_image,wavelet_augmented_image,training_labels_final)=\
   mixup_augmentation(spectrogram_training_data,range_doppler_training_data,wavelet_training_data, training_labels, repeat_of_mixup, alpha=0.2)


  return (spectrogram_augmented_image,range_doppler_augmented_image,wavelet_augmented_image,training_labels_final,\
     spectrogram_validation_data,range_doppler_validation_data,wavelet_validation_data, validation_labels)

In [7]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
wavelet_concat_shuffle = normalize_inputs(wavelet_concat_shuffle, normalize_inputs_enable)

range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [8]:
n_features = range_doppler_concat_shuffle.shape[1]
n_steps = range_doppler_concat_shuffle.shape[2]
range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3))
spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))
wavelet_concat_shuffle = np.transpose(wavelet_concat_shuffle, axes = (0,2,1,3))


In [9]:
t = time.time()
# ---------- Parameters ----------------
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 70
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 256
dense_unit_of_spectrogram_function = 8
dense_unit_of_wavelet=8
decoder_dense_unit = 256
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del spectrogram_concat_shuffle_test
    del range_doppler_training_data
    del range_doppler_concat_shuffle_train
    del spectrogram_concat_shuffle_train
    del spectrogram_training_data

  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    wavelet_concat_shuffle_test = wavelet_concat_shuffle[randomlist_for_test_indx,:,:,:]

    range_doppler_concat_label_shuffle_test = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]


    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    wavelet_concat_shuffle_train =wavelet_concat_shuffle[randomlist_for_train_indx,:,:,:]
    label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]



      # ---------------- Split data train and validation Augmentation ----------------
    (spectrogram_training_data,range_doppler_training_data,wavelet_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data,wavelet_validation_data, validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,wavelet_concat_shuffle_train,label_shuffle_train)

    # ---------------- Neural Network Architecture ----------------



    def lstm_encoder_network_1(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_range_doppler_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def lstm_encoder_network_2(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_spectrogram_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)



    def lstm_encoder_network_3(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_wavelet)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(decoder_dense_unit)(input)
      x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      x = Dropout(0.3)(x)
      x = Dense(dense_size)(x)
      x = LeakyReLU()(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(1, activation="sigmoid")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:3]
    base_network_lstm = lstm_encoder_network_1(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_lstm(range_doppler_input)

    input_shape = spectrogram_concat_shuffle_train.shape[1:3]
    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_lstm_2(spectrogram_input)

    input_shape = wavelet_concat_shuffle_train.shape[1:3]
    base_network_lstm_3 = lstm_encoder_network_3(input_shape)
    wavelet_input  = Input(shape=input_shape)
    processed_wavelet = base_network_lstm_3(wavelet_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram, processed_wavelet])


    base_decoder_network = decoder_for_concat((concat_layer.shape[1],))
    out = base_decoder_network(concat_layer)


    model = Model(inputs=[range_doppler_input, spectrogram_input,wavelet_input], outputs=[out])


    if repeat_run_number == 0:
      print(base_network_lstm.summary())
      print(base_network_lstm_2.summary())
      print(base_decoder_network.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')

    history = model.fit((range_doppler_training_data, spectrogram_training_data,wavelet_training_data),(training_labels),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((range_doppler_validation_data, spectrogram_validation_data,wavelet_validation_data) , (validation_labels)))

    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test,wavelet_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],batch_size=batch_size)
    gc.collect()
    # ---------------- Get Test Results ----------------

    y_test_predicted = model.predict((range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test,wavelet_concat_shuffle_test), batch_size=batch_size)


    # ----- Binarize y_test_predicted values -----
    y_test_predicted_binary = np.zeros(y_test_predicted.size)
    for ii in range(y_test_predicted.size):
      if y_test_predicted[ii] < 0.5:
        y_test_predicted_binary[ii] = 0
      else:
        y_test_predicted_binary[ii] = 1

    test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test, y_test_predicted_binary, average='macro')

    test_accuracy_per_fold.append(test_accuracy)
    f1_score_per_fold.append(test_f1_score)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  print(f1_score_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 32)        │        14,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 3200)           │        12,800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 848,256 (3.24 MB)

 Trainable params: 841,344 (3.21 MB)

 Non-trainable params: 6,912 (27.00 KB)

...

 Trainable params: 86,913 (339.50 KB)

 Non-trainable params: 512 (2.00 KB)

None
Epoch 1/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 112ms/step - accuracy: 0.2963 - loss: 0.9733 - val_accuracy: 0.7222 - val_loss: 0.6207
Epoch 2/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.3010 - loss: 0.8329 - val_accuracy: 0.6667 - val_loss: 0.5049
Epoch 3/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.4129 - loss: 0.5973 - val_accuracy: 0.7778 - val_loss: 0.4408
Epoch 4/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.4556 - loss: 0.5829 - val_accuracy: 0.7778 - val_loss: 0.3517
Epoch 5/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.4824 - loss: 0.5505 - val_accuracy: 0.7778 - val_loss: 0.3328
Epoch 6/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.5054 - loss: 0.5109 - val_accuracy: 0.7778 - val_loss: 0.3397
Epoch 7/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.5046 - loss: 0.4349 - val_accuracy: 0.8333 - val_loss: 0.3086
Epoch 8/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.5389 - loss: 0.3812 - val_accuracy: 0.77

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 740ms/step
[0.9039525628089905]
[0.9036691134952004]
Epoch 1/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 109ms/step - accuracy: 0.3025 - loss: 0.7630 - val_accuracy: 1.0000 - val_loss: 0.3079
Epoch 2/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3981 - loss: 0.6807 - val_accuracy: 0.9444 - val_loss: 0.2832
Epoch 3/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3800 - loss: 0.5980 - val_accuracy: 0.9444 - val_loss: 0.1977
Epoch 4/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4197 - loss: 0.5770 - val_accuracy: 0.9444 - val_loss: 0.1739
Epoch 5/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4819 - loss: 0.5348 - val_accuracy: 0.8889 - val_loss: 0.2101
Epoch 6/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4922 - loss: 0.4702 - val_accuracy: 0.8889 - val_loss: 0.2052
Epoch 7/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5294 - loss: 0.4247 - val_accuracy: 0.8889 - val_loss: 0.2334
Epoch 8/70
14/14 ━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 794ms/step
Epoch 1/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 108ms/step - accuracy: 0.2547 - loss: 0.9302 - val_accuracy: 0.7778 - val_loss: 0.4835
Epoch 2/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.3824 - loss: 0.6423 - val_accuracy: 0.7778 - val_loss: 0.5404
Epoch 3/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.4035 - loss: 0.6032 - val_accuracy: 0.7778 - val_loss: 0.4744
Epoch 4/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.3884 - loss: 0.5489 - val_accuracy: 0.7778 - val_loss: 0.4683
Epoch 5/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.3995 - loss: 0.5492 - val_accuracy: 0.8333 - val_loss: 0.4520
Epoch 6/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4276 - loss: 0.5219 - val_accuracy: 0.8889 - val_loss: 0.4856
Epoch 7/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4770 - loss: 0.4143 - val_accuracy: 0.8889 - val_loss: 0.5145
Epoch 8/70
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4759 -

In [10]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Mean test accuracy is 0.886, mean test f1 score is 0.886, max test accuracy is 0.904, max test f1 score is 0.904, min test accuracy is 0.860, min test f1 score is 0.860, std of test accuracy is 0.019, std of test f1 score is 0.019
Time elapsed through all process: 570.433, sec
